# Benchmarking NetworkX compatibility
# Skip notebook test
This notebook benchmark the use of a NetworkX Graph object as input into algorithms.  <p>
The intention of the feature is to be able to drop cuGraph into existing NetworkX code in spot where performance is not optimal.


### Betweenness Centrality
Both NetworkX and cuGraph allow for estimating the betweenness centrality score by using a subset of vertices rather than all the vertices.  WHile that does produce a less accurate answer, it dramatically improves performance when the sample is small.  For this test, the algorithms will use only 10% of the vertices to compute the estimate 


__Notebook Credits__

* Original Authors: Bradley Rees
* Last Edit: 09/27/2020

RAPIDS Versions: 0.16

Test Hardware
```
    GV100 32G, CUDA 10,0
    Intel(R) Core(TM) CPU i7-7800X @ 3.50GHz
    32GB system memory
```

In [ ]:
import networkx as nx
import cugraph as cnx
import time
import operator

In [ ]:
# starting number of Nodes
N = 100

In [ ]:
# average degree
M = 16

In [ ]:
def run_nx(G, k=None):
    t1 = time.time()
    bc = nx.betweenness_centrality(G, k)
    t2 = time.time() - t1
    return t2, bc

In [ ]:
def run_cu(G, k=None):
    t1 = time.time()
    bc = cnx.betweenness_centrality(G, k)
    t2 = time.time() - t1
    return t2, bc

In [ ]:
print("Betweenness Cenytrality - use all nodes - limit scale to 3,200 nodes so that executing time is not days")
print(f"Node \tEdges  \tSpeedup  \t\tcreate time  \t\tnx time  \t\tcu time ")

for x in range(6):
    if x == 0:
        n = N
    else:
        n = n * 2

    
    t1 = time.time()    
    # create a random graph
    G = nx.barabasi_albert_graph(n, M)
    g_time = time.time() - t1
    
    num_edges = G.number_of_edges()
    num_nodes = G.number_of_nodes()
    
    time_nx, bc = run_nx(G)
    time_cu, bcc = run_cu(G)

    speedup = time_nx / time_cu
    print(f"{num_nodes}\t{num_edges}\t{speedup}\t{g_time}\t{time_nx}\t{time_cu}")
    

In [ ]:
print("PageRank - run larger graph since algorithm is fast")
print(f"Node \tEdges  \tSpeedup \t\tnx time  \t\tcu time ")

pr_speedup = []

for x in range(15):
    if x == 0:
        n = N
    else:
        n = n * 2

    # create a random graph
    G = nx.barabasi_albert_graph(n, M)
    num_edges = G.number_of_edges()
    num_nodes = G.number_of_nodes()
    
    t1 = time.time()    
    nx_pr = nx.pagerank(G)
    time_nx = time.time() - t1
    
    t1 = time.time()    
    cp_pr = cnx.pagerank(G)
    time_cu = time.time() - t1

    speedup = time_nx / time_cu
    print(f"{num_nodes}\t{num_edges}  \t{speedup}\t{time_nx}\t{time_cu}")
    

___
Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___